In [11]:
import numpy as np
import numpy.linalg as la

In [12]:
data = np.array([[1, 2, 1, 2, 10, 10.3, 10.5, 10.7],
                 [1, 1, 2, 2,  2,  2,  2, 2]])
labels = np.array([[-1, -1, 1, 1, 1, 1, 1, 1]])
blue_th = np.array([[0, 1]]).T
blue_th0 = -1.5
red_th = np.array([[1, 0]]).T
red_th0 = -2.5

In [13]:
def magnitude(th):
    return la.norm(th)

def margin(x, y, th, th0):
    return y*(th.T@x+th0)/magnitude(th)

In [14]:
def score_sum(data, labels, th, th0):
    _, n = data.shape
    margin_sum = 0
    for i in range(n):
        margin_sum += margin(data[:, i:i+1], labels[:, i:i+1], th, th0)
    return margin_sum[0, 0]

def score_min(data, labels, th, th0):
    _, n = data.shape
    margin_min = margin(data[:, 0:1], labels[:, 0:1], th, th0)[0, 0]
    for i in range(1, n):
        margin_min = min(margin_min, margin(data[:, i:i+1], labels[:, i:i+1], th, th0)[0, 0])
    return margin_min

def score_max(data, labels, th, th0):
    _, n = data.shape
    margin_max = margin(data[:, 0:1], labels[:, 0:1], th, th0)[0, 0]
    for i in range(1, n):
        margin_max = max(margin_max, margin(data[:, i:i+1], labels[:, i:i+1], th, th0)[0, 0])
    return margin_max


In [15]:
print(score_sum(data, labels, red_th, red_th0))
print(score_min(data, labels, red_th, red_th0))
print(score_max(data, labels, red_th, red_th0))

31.5
-1.5
8.2


In [16]:
print(score_sum(data, labels, blue_th, blue_th0))
print(score_min(data, labels, blue_th, blue_th0))
print(score_max(data, labels, blue_th, blue_th0))

4.0
0.5
0.5


In [17]:
data = np.array([[1.1, 1, 4],[3.1, 1, 2]])
labels = np.array([[1, -1, -1]])
th = np.array([[1, 1]]).T
th0 = -4

In [18]:
def hinge_loss(x, y, th, th0, ref):
    m = margin(x, y, th, th0)
    if m < ref:
        return 1 - m/ref
    return 0

In [19]:
for i in range(3):
    print(hinge_loss(data[:, i:i+1], labels[:, i:i+1], th, th0, 1/2**0.5))

[[0.8]]
0
[[3.]]


In [20]:
def cv(arr):
    return np.array([arr]).T

In [21]:
def f1(x):
    return float((2 * x + 3)**2)

def df1(x):
    return 2 * 2 * (2 * x + 3)

def f2(v):
    x = float(v[0]); y = float(v[1])
    return (x - 2.) * (x - 3.) * (x + 3.) * (x + 1.) + (x + y -1)**2

def df2(v):
    x = float(v[0]); y = float(v[1])
    return cv([(-3. + x) * (-2. + x) * (1. + x) + \
               (-3. + x) * (-2. + x) * (3. + x) + \
               (-3. + x) * (1. + x) * (3. + x) + \
               (-2. + x) * (1. + x) * (3. + x) + \
               2 * (-1. + x + y),
               2 * (-1. + x + y)]).T

In [22]:
def package_ans(gd_vals):
    x, fs, xs = gd_vals
    return [x.tolist(), [fs[0], fs[-1]], [xs[0].tolist(), xs[-1].tolist()]]

In [31]:
def gd(f, df, x0, step_size_fn, max_iter):
    x = x0.copy()
    xs = []
    fs = []
    xs.append(x0)
    fs.append(f(x0))
    for i in range(max_iter):
        x = x-step_size_fn(i)*df(x)
        xs.append(x)
        fs.append(f(x))
    return (x, fs, xs)
    

In [40]:
def num_grad(f, delta=0.001):
    def df(x):
        n = x.shape[0]
        ret = []
        diff = np.zeros((n, 1))
        for i in range(n):
            delta0 = np.copy(diff)
            delta0[i, 0] = delta
            ret.append(((f(x+delta0)-f(x-delta0))/(2*delta)))
        return cv(ret)
    return df

In [43]:
def minimize(f, x0, step_size_fn, max_iter):
    df = num_grad(f, delta=0.001)
    return gd(f, df, x0, step_size_fn, max_iter)

In [44]:
ans = package_ans(minimize(f1, cv([0.]), lambda i: 0.1, 1000))

C:\Users\jiama\AppData\Local\Temp\ipykernel_25388\3569026143.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float((2 * x + 3)**2)


In [45]:
# v = y(theta*x+theta_0)
def hinge(v):
    return max(0, 1-v)

# x is dxn, y is 1xn, th is dx1, th0 is 1x1
def hinge_loss(x, y, th, th0):
    loss = 0
    n = x.shape[1]
    for i in range(n):
        loss += hinge((y[:, i:i+1]*(th.T@x[:, i:i+1]+th0))[0, 0])
    return loss

# x is dxn, y is 1xn, th is dx1, th0 is 1x1, lam is a scalar
def svm_obj(x, y, th, th0, lam):
    n = x.shape[1]
    return hinge_loss(x, y, th, th0)/n + lam*la.norm(th)**2

In [46]:
def super_simple_separable():
    X = np.array([[2, 3, 9, 12],
                  [5, 2, 6, 5]])
    y = np.array([[1, -1, 1, -1]])
    return X, y

sep_e_separator = np.array([[-0.40338351], [1.1849563]]), np.array([[-2.26910091]])

# Test case 1
x_1, y_1 = super_simple_separable()
th1, th1_0 = sep_e_separator
ans = svm_obj(x_1, y_1, th1, th1_0, .1)

# Test case 2
ans = svm_obj(x_1, y_1, th1, th1_0, 0.0)

In [69]:
def d_hinge(v):
    return np.where(v >= 1, 0, -1)

def d_hinge_loss_th(x, y, th, th0):
    return d_hinge(y*(np.dot(th.T, x) + th0))*y*x

def d_hinge_loss_th0(x, y, th, th0):
    return d_hinge(y*(np.dot(th.T, x) + th0)) * y

def d_svm_obj_th(x, y, th, th0, lam):
    return np.mean(d_hinge_loss_th(x, y, th, th0), axis = 1, keepdims = True) + lam * 2 * th

def d_svm_obj_th0(x, y, th, th0, lam):
    return np.mean(d_hinge_loss_th0(x, y, th, th0), axis = 1, keepdims = True)

def svm_obj_grad(X, y, th, th0, lam):
    grad_th = d_svm_obj_th(X, y, th, th0, lam)
    grad_th0 = d_svm_obj_th0(X, y, th, th0, lam)
    return np.vstack([grad_th, grad_th0])

In [67]:
X1 = np.array([[1, 2, 3, 9, 10]])
y1 = np.array([[1, 1, 1, -1, -1]])
th1, th10 = np.array([[-0.31202807]]), np.array([[1.834     ]])
X2 = np.array([[2, 3, 9, 12],
               [5, 2, 6, 5]])
y2 = np.array([[1, -1, 1, -1]])
th2, th20=np.array([[ -3.,  15.]]).T, np.array([[ 2.]])

d_hinge(np.array([[ 71.]])).tolist()
d_hinge(np.array([[ -23.]])).tolist()
d_hinge(np.array([[ 71, -23.]])).tolist()

d_hinge_loss_th(X2[:,0:1], y2[:,0:1], th2, th20).tolist()
d_hinge_loss_th(X2, y2, th2, th20).tolist()
d_hinge_loss_th0(X2[:,0:1], y2[:,0:1], th2, th20).tolist()
d_hinge_loss_th0(X2, y2, th2, th20).tolist()

d_svm_obj_th(X2[:,0:1], y2[:,0:1], th2, th20, 0.01).tolist()
d_svm_obj_th(X2, y2, th2, th20, 0.01).tolist()
d_svm_obj_th0(X2[:,0:1], y2[:,0:1], th2, th20, 0.01).tolist()
d_svm_obj_th0(X2, y2, th2, th20, 0.01).tolist()

svm_obj_grad(X2, y2, th2, th20, 0.01).tolist()
svm_obj_grad(X2[:,0:1], y2[:,0:1], th2, th20, 0.01).tolist()

[[-0.06], [0.3], [0.0]]

In [78]:
np.mean(d_hinge_loss_th(X2, y2, th2, th20)*y2*X2, axis=1, keepdims=True) + 0.02*th2

array([[3.69],
       [2.05]])